In [16]:
# Função para conversão de byte code para inteiro
def toInteger(octaBin):
    intNumber = 0
    octaBin = list(octaBin)
    for i in range(len(octaBin)):
        octaBin[i] = int(octaBin[i])
    intNumber += pow((octaBin[0] * 2),7)
    intNumber += pow((octaBin[1] * 2),6)
    intNumber += pow((octaBin[2] * 2),5)
    intNumber += pow((octaBin[3] * 2),4)
    intNumber += pow((octaBin[4] * 2),3)
    intNumber += pow((octaBin[5] * 2),2)
    intNumber += pow((octaBin[6] * 2),1)
    intNumber += octaBin[7]
    return(intNumber)

In [17]:
# Importacao da biblioteca
from PIL import Image
# Inicializacao das variáveis
im = Image.open("/mnt/d/Google Drives/fernando.dipp/Análise e Desenvolvimento de Sistemas/2022-02/S096 - Trabalho de Conclusao de Curso/Esteganografia/Algoritmo/GOPR2271.jpg")
mensagem = "Meu nome e Fernandoo"
coordX = 0
coordY = 0
maxX = im.size[0]
maxY = im.size[1]

# Iterando cada letra da mensagem a ser ocultada, disparando a rotina abaixo
for i in range(0, len(mensagem)):
    # transformando a letra em bits (tabela ASCII)
    octaChar = '{0:08b}'.format(ord(mensagem[i]),'b')
    octaChar = list(octaChar)
    # variáveis de controle
    k = 0
    charCounter = 0
    # Iterando cada bit do byte ASCII
    for j in range(0,7,3):
        # Coletando os níveis RGB do determinado pixel
        r, g, b = im.getpixel((coordX+k,coordY))
        # Convertendo de inteiro para binário (1 byte)
        octaR = '{0:08b}'.format(r,'b')
        octaG = '{0:08b}'.format(g,'b')
        octaB = '{0:08b}'.format(b,'b')
        # Convertendo o bit menos significativo da cor Vermelha
        octaR = list(octaR)
        octaR[-1] = octaChar[charCounter]
        octaR = ''.join(octaR)
        # Convertendo o bit menos significativo da cor Verde
        octaG = list(octaG)
        octaG[-1] = octaChar[charCounter+1]
        octaG = ''.join(octaG)
        octaB = list(octaB)
        try:
            # Convertendo o bit menos significativo da cor Azul
            octaB[-1] = octaChar[charCounter+2]
        except:
            # Aplicando a flag de final de mensagem
            if (i >= len(mensagem)-1):
                octaB[-1] = '1'
            # Aplicando a flag de continuação da mensagem
            else:
                octaB[-1] = '0'
        
        octaB = ''.join(octaB)
        # Atualizando os níveis das cores RGB da foto
        im.putpixel((coordX+k,coordY),(toInteger(octaR),toInteger(octaG),toInteger(octaB)))
        # Incrementando as variáveis de controle
        charCounter += 3
        k += 1
    coordX += 3
    if (coordX >= maxX -1):
        coordY += 1
        coordX = 0

# Salvando a foto manipulada com a mensagem ocultada
im.save("/mnt/d/estegano.tiff", quality=100, compress_level=0, subsampling=0,format='TIFF')

In [18]:
# Importacao da biblioteca
from PIL import Image
# Inicializacao das variáveis
im = Image.open("/mnt/d/estegano.tiff")
segredo = ''
coordX = 0
coordY = 0
maxX = im.size[0]
maxY = im.size[1]

controle = True
while controle != False:
    byteChar = []
    r, g, b = im.getpixel((coordX,coordY))    
    octaR = '{0:08b}'.format(r,'b')[-1]
    byteChar.append(octaR)
    octaG = '{0:08b}'.format(g,'b')[-1]
    byteChar.append(octaG)
    octaB = '{0:08b}'.format(b,'b')[-1]
    byteChar.append(octaB)
    r, g, b = im.getpixel((coordX+1,coordY))    
    octaR = '{0:08b}'.format(r,'b')[-1]
    byteChar.append(octaR)
    octaG = '{0:08b}'.format(g,'b')[-1]
    byteChar.append(octaG)
    octaB = '{0:08b}'.format(b,'b')[-1]
    byteChar.append(octaB)  
    r, g, b = im.getpixel((coordX+2,coordY))    
    octaR = '{0:08b}'.format(r,'b')[-1]
    byteChar.append(octaR)
    octaG = '{0:08b}'.format(g,'b')[-1]
    byteChar.append(octaG)
    octaB = '{0:08b}'.format(b,'b')[-1]
    if (octaB == '1'):
        controle = False        
    if (octaB != '1'):      
        byteCodeChar = ''.join(byteChar)
        segredo += chr(toInteger(byteCodeChar))
        byteChar.append(octaB)

    coordX += 3
print('a mensagem é',segredo)
            


a mensagem é Meu nome e Fernando
